In [1]:
import sys
import os
sys.path.append('..')

import random

import numpy as np
import pandas as pd
import torch
import transformers
import wandb

from transformers import PatchTSTConfig, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import PatchTSTForPrediction as PatchTSTForPredictionOG
from TimeSeriesJEPA.models import PatchTSTModelJEPA, PatchTSTForPrediction
from TimeSeriesJEPA.datasets.benchmark_dataset import BenchmarkDataset
os.environ["WANDB_PROJECT"] = "TimeSeriesJEPA" 

In [2]:
csv_path = r"D:\Coursework\MTS\dataset\ETT-small\ETTh1.csv"
dataset = "ETTh1"
num_workers = 4  # Reduce this if you have low number of CPU cores
batch_size = 10  # Reduce if not enough GPU memory available
context_length = 512
forecast_horizon = 96
patch_length = 8
patch_stride = 8
num_input_channels=7
d_model=64
num_attention_heads=4
num_hidden_layers=3
ffn_dim=64
dropout=0.05
head_dropout=0.2
pooling_type=None
channel_attention=False
scaling="std"
loss="mse"
pre_norm=True
norm_type="batchnorm"
positional_encoding_type = "sincos"

learning_rate=0.0001
num_train_epochs=30
max_steps=200

setting = 'Supervised_{}_PatchTST_sl{}_dm{}_nh{}_el{}_fd{}_bs{}_lr{}_e{}'.format(
        dataset,
        context_length,
        d_model,
        num_attention_heads,
        num_hidden_layers,
        ffn_dim,
        batch_size,
        learning_rate,
        num_train_epochs
        )

In [3]:

trainwindowds = BenchmarkDataset(csv_path=csv_path, context_length=context_length, prediction_length=forecast_horizon, flag='train', returndict=True)
valwindowds = BenchmarkDataset(csv_path=csv_path, context_length=context_length, prediction_length=forecast_horizon, flag='test', returndict=True)
print("dataset loaded, total size: ", len(trainwindowds), len(valwindowds))

Total data size:  (17420, 7)
Total data size:  (17420, 7)
dataset loaded, total size:  8033 2785


In [ ]:
print("Loading prediction model")

config = PatchTSTConfig(
    do_mask_input=False,
    context_length=context_length,
    patch_length=patch_length,
    num_input_channels=num_input_channels,
    patch_stride=patch_stride,
    prediction_length=forecast_horizon,
    d_model=d_model,
    num_attention_heads=num_attention_heads,
    num_hidden_layers=num_hidden_layers,
    ffn_dim=ffn_dim,
    dropout=dropout,
    head_dropout=head_dropout,
    pooling_type=pooling_type,
    channel_attention=channel_attention,
    scaling=scaling,
    loss=loss,
    pre_norm=pre_norm,
    norm_type=norm_type,
    positional_encoding_type = positional_encoding_type
)

model = PatchTSTForPredictionOG(config=config)

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model.parameters()])
print("encoder parameters: ", params)

In [ ]:
wandb.init(project="TimeSeriesJEPA", name=setting)


train_args = TrainingArguments(
    output_dir=os.path.join("checkpoints", setting),
    overwrite_output_dir=True,
    learning_rate=learning_rate,
    # num_train_epochs=num_train_epochs,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=1,
    max_steps=max_steps,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=1,  # num_workers,
    logging_strategy="steps",
    logging_steps=1,
    save_total_limit=3,
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"],
    report_to="wandb",
    run_name=setting
    )

# # Create a new early stopping callback with faster convergence properties
# early_stopping_callback = EarlyStoppingCallback(
#     early_stopping_patience=15,  # Number of epochs with no improvement after which to stop
#     early_stopping_threshold=0.001,  # Minimum improvement required to consider as improvement
# )

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=trainwindowds,
    eval_dataset=valwindowds,
    # callbacks=[early_stopping_callback],
)

In [ ]:
print("\n\nDoing forecasting training")
trainer.train()

In [ ]:
trainer.evaluate(valwindowds)
wandb.finish()

In [4]:
print("Loading pretrained encoder model")
setting = "PatchTST_Time300B_sl512_enc_dm64_nh4_el3_fd64_pred_dm32_nh2_el1_fd32_bs256_lr0.0001_pe10_data_nenc1_npred4_ema_05_1"
encoder_model = PatchTSTModelJEPA.from_pretrained("D:\\Coursework\\MTS\\timeseriesJEPA\\results\\"+setting+"\\checkpoint-57120")
print("Done")
encoder_model.cuda()
run_name = "Finetuned_"+dataset+"_"+setting+"_17136"

Loading pretrained encoder model
Done


In [5]:
run_name

'Finetuned_ETTh1_PatchTST_Time300B_sl512_enc_dm64_nh4_el3_fd64_pred_dm32_nh2_el1_fd32_bs256_lr0.0001_pe10_data_nenc1_npred4_ema_05_1_17136'

In [6]:
print("Loading prediction model")

config = PatchTSTConfig(
    do_mask_input=False,
    context_length=context_length,
    patch_length=patch_length,
    num_input_channels=num_input_channels,
    patch_stride=patch_length,
    prediction_length=forecast_horizon,
    d_model=64,
    num_attention_heads=4,
    # num_hidden_layers=4,
    ffn_dim=128,
    dropout=0.05,
    head_dropout=head_dropout,
    pooling_type=None,
    channel_attention=False,
    scaling="std",
    loss="mse",
    pre_norm=True,
    norm_type="batchnorm",
    positional_encoding_type = "sincos"
)

model = PatchTSTForPrediction(config=config, encoder_model=encoder_model)

Loading prediction model


In [7]:
encoder_model

PatchTSTModelJEPA(
  (scaler): PatchTSTScaler(
    (scaler): PatchTSTStdScaler()
  )
  (patchifier): PatchTSTPatchify()
  (encoder): PatchTSTEncoder(
    (embedder): PatchTSTEmbedding(
      (input_embedding): Linear(in_features=8, out_features=64, bias=True)
    )
    (positional_encoder): PatchTSTPositionalEncoding(
      (positional_dropout): Identity()
    )
    (layers): ModuleList(
      (0-2): 3 x PatchTSTEncoderLayer(
        (self_attn): PatchTSTAttention(
          (k_proj): Linear(in_features=64, out_features=64, bias=True)
          (v_proj): Linear(in_features=64, out_features=64, bias=True)
          (q_proj): Linear(in_features=64, out_features=64, bias=True)
          (out_proj): Linear(in_features=64, out_features=64, bias=True)
        )
        (dropout_path1): Identity()
        (norm_sublayer1): PatchTSTBatchNorm(
          (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (ff): Sequential(
          (0):

In [8]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print("encoder parameters: ", params)

encoder parameters:  393312


In [9]:
params = sum([np.prod(p.size()) for p in encoder_model.parameters()])
print("encoder parameters: ", params)

encoder parameters:  80448


In [10]:
wandb.init(project="TimeSeriesJEPA", name=run_name)

train_args_jepa = TrainingArguments(
    output_dir=os.path.join("checkpoints", run_name),
    overwrite_output_dir=True,
    learning_rate=learning_rate,
    # num_train_epochs=num_train_epochs,
    max_steps=max_steps,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=1,  # num_workers,
    logging_strategy="steps",
    logging_steps=1,
    save_total_limit=3,
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"],
    report_to="wandb",
    run_name=run_name
)


trainer_jepa = Trainer(
    model=model,
    args=train_args_jepa,
    train_dataset=trainwindowds,
    eval_dataset=valwindowds,
)

wandb: Currently logged in as: vg2523 (hpml_4). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


c:\Users\Omen\anaconda3\envs\mts\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [11]:
print("\n\nDoing forecasting training")
trainer_jepa.train()



Doing forecasting training


  0%|          | 0/200 [00:00<?, ?it/s]

{'loss': 1.6188, 'grad_norm': 7.519554615020752, 'learning_rate': 9.95e-05, 'epoch': 0.0}


  0%|          | 0/279 [00:00<?, ?it/s]

{'eval_loss': 1.607560396194458, 'eval_runtime': 7.9619, 'eval_samples_per_second': 349.79, 'eval_steps_per_second': 35.042, 'epoch': 0.0}
{'loss': 1.4725, 'grad_norm': 6.31898832321167, 'learning_rate': 9.900000000000001e-05, 'epoch': 0.0}


...

  0%|          | 0/279 [00:00<?, ?it/s]

{'eval_loss': 0.533547043800354, 'eval_runtime': 11.408, 'eval_samples_per_second': 244.127, 'eval_steps_per_second': 24.457, 'epoch': 0.25}
{'loss': 0.6493, 'grad_norm': 1.5918866395950317, 'learning_rate': 0.0, 'epoch': 0.25}


  0%|          | 0/279 [00:00<?, ?it/s]

{'eval_loss': 0.5332947373390198, 'eval_runtime': 11.1541, 'eval_samples_per_second': 249.685, 'eval_steps_per_second': 25.013, 'epoch': 0.25}
{'train_runtime': 3094.2755, 'train_samples_per_second': 0.646, 'train_steps_per_second': 0.065, 'train_loss': 0.7286773881316185, 'epoch': 0.25}


TrainOutput(global_step=200, training_loss=0.7286773881316185, metrics={'train_runtime': 3094.2755, 'train_samples_per_second': 0.646, 'train_steps_per_second': 0.065, 'total_flos': 20375470080000.0, 'train_loss': 0.7286773881316185, 'epoch': 0.24875621890547264})

In [13]:
trainer_jepa.evaluate(valwindowds)
wandb.finish()

  0%|          | 0/279 [00:00<?, ?it/s]

Error: You must call wandb.init() before wandb.log()